# **Soni - do**
## **Generating Music with Machine Learning**


#### Author: Sonia Cobo
#### Date: July 2021

### Though this project doesn't have a hypothesis per se, it was done to kind off prove how AI has advanced and it is now able to generate music which has been associated with emotions and human capabilities for a long period of time.

In [12]:
# data augmentation - dividir canciones, modificarlas para tener mas datos

# Data

### The input to the model will be a series of notes from a MIDI file. MIDI (Musical Instrument Digital Interface) is a technical standard that describes a communications protocol, digital interface, and electrical connectors that connect a wide variety of electronic musical instruments and computers. They don't contain actual audio data and are small in size. They explain what notes are played, when they're played, and how long or loud each note should be.

### To keep the project simple only files with one instrument were chosen, in this case the instrument is piano and the type of songs is classical. 
### These songs have been obtained from the following datasets: http://www.piano-midi.de/ and https://www.mfiles.co.uk/classical-midi.htm


In [13]:
# no descargardas aun: https://github.com/Skuldur/Classical-Piano-Composer/tree/master/midi_songs
# https://drive.google.com/file/d/1qnQVK17DNVkU19MgVA4Vg88zRDvwCRXw/view

### Import all libraries

In [151]:
# data manipulation
import numpy as np
import pandas as pd 
from random import randint
from sqlalchemy import create_engine

# manipulate midi files
import glob
from music21 import *
#from music21 import converter, instrument, note, chord, meter, stream, duration, corpus
import pygame

# visualization
import seaborn as sns
import matplotlib.pyplot as plt

# route files
import os
import sys

# ml model
import pickle

import tensorflow as tf
from tensorflow import keras

from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation
from keras.layers import BatchNormalization 
from keras.callbacks import ModelCheckpoint
from keras.layers import Reshape
from keras.layers import LeakyReLU
from keras.layers import Conv2D
from keras.layers import Conv1D
from keras.layers import Flatten
from keras.layers import Conv2DTranspose
from keras.layers import Conv1DTranspose

# my libraries
from utils.sql_tb import MySQL



### Paths

In [152]:
# The route of this file is added to the sys path to be able to import/export functions
sep = os.sep
def route (steps):
    """
    This function appends the route of the file to the sys path
    to be able to import files from/to other foders within the EDA project folder.
    """
    route = os.getcwd()
    for i in range(steps): 
        route = os.path.dirname(route)
    sys.path.append(route)
    return route

In [153]:
# paths

# path to raw data
path = route(1) + sep + "data" + sep + "raw_data" + sep
# path to data in the right key
path_1 = route(1) + sep + "data" + sep + "converted_data" + sep
# path to compiled notes list
path_2 = route(1) + sep + "data" + sep + "notes" + sep
# path to generated models
path_3 = route(1) + sep + "models" + sep
# path to generated midi files
path_4 = route(1) + sep + "reports" + sep

## Midi file exploration

In [154]:
# All information from the midi file (i.e. notes, pitch, chord, time signature, etc) is contained within the component list

def info_midi (path, filename):
    """
    It returns all midi file information given its path and filename

    """
    # Convert to Score object
    file = converter.parse(path + filename)
    components = []
    # read file information
    for element in file.recurse():  
        components.append(element)
    return components

components = info_midi(path, "alb_esp1.mid")
#components

## Data preparation

In [155]:
# Each midi file contains notes and chords. These two properties will be the input and output of the LSTM network so 
# they need to be taken out from all midi files. 

def get_notes_per_song(path, filename, save_path, save_name):
    """
    This function extracts all the notes, rests and chords from one midi file
    and saves it in a list in the converted_data folder.

    Param: Path of the midi file, filename (str)
    """
    components = info_midi(path, filename)
    note_list = []
    
    for element in components:
        # note pitches are extracted
        if isinstance(element, note.Note):
            note_list.append(str(element.pitch))
        # chords are extracted
        elif isinstance(element, chord.Chord):
            note_list.append(".".join(str(n) for n in element.normalOrder))    
        # rests are extracted
        elif isinstance(element, note.Rest):
            note_list.append("NULL")    #further transformation needs this value as str rather than np.nan

    # save list with all componenets extracted
    with open(save_path + save_name, "wb") as filepath:
        pickle.dump(note_list, filepath)
    
    return note_list

In [156]:
one_song = get_notes_per_song(path_1, "C_alb_esp1.mid", path_2, "one_notes")

In [157]:
def get_all_notes(path, save_name, save_path):
    """
    This function extracts all the notes, rests and chords from all midi files 
    and saves it in a list in the converted_data folder.

    Param: Path of the midi file     
    """
    all_notes = []
    list_path = os.listdir(path)
    for filename in list_path:
        output = get_notes_per_song(path, filename, save_path, save_name)
        all_notes += output
        
    return all_notes

In [158]:
all_notes = get_all_notes(path = path_1, save_path = path_2, save_name = "all_notes")

In [159]:
# Load notes and chords previously separated
def load_notes (path, filename):
    """
    Load the note list containing pitches, rests and chords.
    
    Param: Path of the saved note list, and its name as string
    """
    with open(path + filename, "rb") as f:
        loaded_notes = pickle.load(f)
        return loaded_notes

In [160]:
load_chopin = load_notes(path_2, "notes_chopin")


In [162]:
len(load_chopin)

1522

In [221]:
def prepare_sequences(notes, min_note_occurence, sequence_length, step):
    """ 
    This function creates the input and output sequences used by the neural network.
    It returns the x and y of the model.

    Param: 
        Note: List containing all notes, rests and chords
        Sequence_length: Lenght of notes given to the model to help predict the next
        Step: Step (int) between one input sequence and the next one
    """
    
    # get all pitchnames
    pitchnames = sorted(set(notes))
    print('Total unique notes:', len(pitchnames))

    # Calculate occurence
    note_freq = {}
    for elem in notes:
        note_freq[elem] = note_freq.get(elem, 0) + 1

    ignored_notes = set()
    for k, v in note_freq.items():
        if note_freq[k] < min_note_occurence:
            ignored_notes.add(k)
    
    
    print('Unique words before ignoring:', len(pitchnames))
    print('Ignoring words with occurence <', min_note_occurence)
    pitchnames = sorted(set(pitchnames) - ignored_notes)
    print('Unique words after ignoring:', len(pitchnames))

    # create a dictionary to convert pitches (strings) to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))  # rests are included  

    network_input = []
    network_output = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - sequence_length, step): 
        # remove ignored notes from the note list   
        if len(set(notes[i: i+ sequence_length + 1]).intersection(ignored_notes)) == 0:
            network_input.append(notes[i:i + sequence_length])
            network_output.append(notes[i + sequence_length])
    # array of zeros
    x = np.zeros((len(network_input), sequence_length, len(pitchnames)))
    y = np.zeros((len(network_input), len(pitchnames)))
    # exchange note values for their integer-code
    for i, sequence in enumerate(network_input):
        for j, note in enumerate(sequence):
            x[i, j, note_to_int[note]] = 1
        y[i, note_to_int[network_output[i]]] = 1

    return x, y

In [222]:
x, y = prepare_sequences(notes=load_chopin, min_note_occurence=1, sequence_length=100, step=3)  # length y step pueden variar  


Total unique notes: 109
Unique words before ignoring: 109
Ignoring words with occurence < 1
Unique words after ignoring: 109


In [223]:
print(x.shape)
print(y.shape)

(474, 100, 109)
(474, 109)


In [224]:
def prepare_sequences_gan(notes, sequence_length, step):
    """ 
    Prepare the sequences used by the neural network 

    """
    
    # get all pitchnames
    pitchnames = sorted(set(notes))
    print('Total unique notes:', len(pitchnames))

    # create a dictionary to convert pitches (strings) to integers
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))  # rests are included  

    network_input = []
    network_output = []

    #sequence_in = []
    #sequence_out = []

    # create input sequences and the corresponding outputs
    for i in range(0, len(notes) - 2*sequence_length, step):    
        network_input.append(notes[i:i + sequence_length])
        network_output.append(notes[i + sequence_length : i + 2*sequence_length])
        # exchange their values for their integer-code

        # network_input.append([note_to_int[elem] for elem in sequence_in])
        # network_output.append(note_to_int[sequence_out])

    x = np.zeros((len(network_input), sequence_length, len(pitchnames)))
    y = np.zeros((len(network_input), sequence_length, len(pitchnames)))
    for i, sequence in enumerate(network_input):
        for j, note in enumerate(sequence):
            x[i, j, note_to_int[note]] = 1
            y[i, j, note_to_int[network_output[i][j]]] = 1

    return x, y

In [225]:
import gc
gc.collect()

38737

In [226]:
x_gan, y_gan = prepare_sequences_gan(notes=load_chopin, sequence_length=100, step=3)  # length y step pueden variar  


Total unique notes: 109


# Creation of the model

In [271]:
(x.shape[0], x.shape[1], x.shape[2])

(474, 100, 109)

In [272]:
x.shape[1]

100

In [282]:
def generator_model(n_nodes= x.shape[0] , latent_dim=(x.shape[1], x.shape[2])):
	model = Sequential()
	model.add(Dense(n_nodes, input_shape = latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Dense(n_nodes))
	model.add(LeakyReLU(alpha=0.2))
	model.add(BatchNormalization())
	model.add(Dense(x.shape[2]))
	model.add(LeakyReLU(alpha=0.2))
	model.add(BatchNormalization())
	model.add(Reshape(latent_dim))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv1DTranspose(n_nodes, 3, padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(Conv1DTranspose(x.shape[2], 3, activation='softmax', padding='same'))	# softmax - n elem = n salidas
	
	return model

In [283]:
g = generator_model(n_nodes= x.shape[0] , latent_dim=(x.shape[1],x.shape[2]))
g.summary()

Model: "sequential_67"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_83 (Dense)             (None, 100, 474)          52140     
_________________________________________________________________
leaky_re_lu_163 (LeakyReLU)  (None, 100, 474)          0         
_________________________________________________________________
dense_84 (Dense)             (None, 100, 474)          225150    
_________________________________________________________________
leaky_re_lu_164 (LeakyReLU)  (None, 100, 474)          0         
_________________________________________________________________
batch_normalization_31 (Batc (None, 100, 474)          1896      
_________________________________________________________________
dense_85 (Dense)             (None, 100, 109)          51775     
_________________________________________________________________
leaky_re_lu_165 (LeakyReLU)  (None, 100, 109)        

In [229]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype("float64")
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [286]:
def generate_notes_gan(notes, model, temperature=1.0):
    """ 
    Generate notes from the neural network based on a sequence of notes 
    """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(notes)-100-1)

    pitchnames = sorted(set(notes))

    note_to_int = dict((note, number) for number, note in enumerate(pitchnames)) 
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    pattern = notes[start: (start+100)] 
    prediction_output = []
    patterns = []

    # generate 500 notes, roughly two minutes of music

    prediction_input = np.zeros((1, 100, len(pitchnames)))
    for j, note in enumerate(pattern):
        prediction_input[0, j, note_to_int[note]] = 1.0
    preds = model.predict(prediction_input, verbose=0)[0]
    for elem in list(preds):
        next_index = sample(elem, temperature=temperature)
        next_note = int_to_note[next_index]
        #pattern = pattern[1:]
        #pattern.append(next_note)
        prediction_output.append(next_note)

        patterns.append(next_index)
        #patterns = patterns[1:len(patterns)]
    return prediction_output, patterns

In [287]:
prediction_output, patterns = generate_notes_gan(load_chopin, g, temperature=1)
print(prediction_output)

['8.11.2.4', 'E-3', '10.2', 'B3', 'C#3', 'A4', '9.2', 'E6', 'G#4', '10.2', 'E-6', '9.0.3', '8.11.2.4', 'C#3', '4', 'B-5', 'C#2', '4.8', '10.1', '2.5', 'G5', 'E-3', 'C#6', 'G4', 'G2', 'A6', 'D4', 'C3', 'G#2', '6.9.1', '2.7', 'A2', '9', '2.6', '9.0.3', '8.11', '1.2', 'D4', '1.6', '5.9.11', '1.6', 'F#3', '9.0.4', '9.11', 'G#5', '1.6', 'E6', 'B-5', '9.0.4', '9.11', '2.7', '9', '8.11', '6.9.1', '2.6', '6.11', '4.7.11', '11.4', '2.5', 'B-4', '9.0', '10.0', 'B-4', 'E1', 'A5', '11.3', 'G#2', '9.0.3', 'E-5', 'F#3', '4.6.10', 'G4', '4.7.10.0', 'C#4', 'B4', '7.0', '1.4', 'A6', '2.4.6.10', 'C#3', 'D3', 'E-4', 'C3', 'C6', 'C#2', '1.6', '4.6.10', 'E3', 'B-4', '10.2', '3.9', 'F4', 'A6', '0.3.6', '2.8', 'E-6', 'C#2', 'G#3', '6.9', 'F#5']


In [ ]:
len(prediction_output)

100

In [569]:
def create_midi(prediction_output, patterns, path):
    """ convert the output from the prediction to notes and create a midi file from the notes"""
    
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif ("NULL" in pattern):
            new_rest = note.Rest(pattern)
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)   
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write("midi", fp= path + "test_output_11_gmodel_chopin.mid")   # first output 01/07/2021

    return midi_stream

In [570]:
create_midi = create_midi(prediction_output, patterns, path_4)


In [571]:
def play_music(music_file):
    """
    Play music given a midi file path
    """
    import music21
    try:
        # allow to stop the piece 
        pygame.mixer.init()
        clock = pygame.time.Clock() 
        pygame.mixer.music.load(music_file)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            # check if playback has finished
            clock.tick(10)

        freq = 44100    # audio CD quality
        bitsize = -16   # unsigned 16 bit
        channels = 2    # 1 is mono, 2 is stereo
        buffer = 1024    # number of samples
        pygame.mixer.init(freq, bitsize, channels, buffer)

    except KeyboardInterrupt:
        while True:
            action = input('Enter Q to Quit, Enter to Skip.').lower()
            if action == 'q':
                pygame.mixer.music.fadeout(1000)
                pygame.mixer.music.stop()
            else:
                break

In [573]:
# Plays music when the cell is executed 

play_music(path_4 + "test_output_11_gmodel_chopin.mid")

In [ ]:
# hasta aquí va!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!

In [288]:
def generate_real_samples(x, n_samples):
    """
    Load and prepare training notes
    """
    # choose random instances
    start = np.random.randint(0, len(x)-100-1)
    # retrieve selected images
    x_real = x[start: (start+n_samples)] 
    # generate 'real' class labels (1)
    y_real = np.ones((n_samples, 1))

    return x_real, y_real

In [289]:
x_real, y_real = generate_real_samples(x, n_samples=100)
y_real.shape

(100, 1)

In [290]:
def generate_latent_points(x, n_samples):
    import random
    # create random matrix of numbers 
    x_latent = np.zeros((n_samples, x.shape[1], x.shape[2]))
  
    for j, elem in enumerate(x_latent):
        for k, row in enumerate(elem):
            num = random.randint(0, x.shape[2])
            for i in range(len(row)):
                if i == num:
                    x_latent[j][k][i] = 1

    return x_latent


In [291]:
x_latent = generate_latent_points(x, n_samples=100)

In [292]:
def generate_fake_data(x, g_model, n_samples):
	# create 'fake' class labels (0)
	y_fake = np.zeros((n_samples, 1))

	# generate points in latent space
	x_latent = generate_latent_points(x, n_samples)
	# predict outputs
	x_fake = g_model.predict(x_latent, verbose=0)

	return x_fake, y_fake


In [293]:
x_fake, y_fake = generate_fake_data(x, g, n_samples=100)

In [294]:
# define the standalone discriminator model
def discriminator_model(num_units, n_inputs=(x.shape[1], x.shape[2])):
	model = Sequential()
	# normal
	model.add(Conv1D(num_units, (3), padding='same', input_shape=n_inputs))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 40x40
	model.add(Conv1D(num_units, (3), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 20x30
	model.add(Conv1D(num_units, (3), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	model.add(BatchNormalization())
	# downsample to 10x10
	model.add(Conv1D(num_units, (3), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# downsample to 5x5
	model.add(Conv1D(num_units, (3), padding='same'))
	model.add(LeakyReLU(alpha=0.2))
	# classifier
	model.add(Flatten())
	model.add(Dropout(0.4))
	model.add(Dense(1, activation='sigmoid'))
	# compile model
	model.compile(loss='binary_crossentropy', optimizer="adam", metrics=['accuracy'])
	return model
	

In [295]:
d = discriminator_model(num_units=x.shape[0])
d.summary()

Model: "sequential_68"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_39 (Conv1D)           (None, 100, 474)          155472    
_________________________________________________________________
leaky_re_lu_168 (LeakyReLU)  (None, 100, 474)          0         
_________________________________________________________________
conv1d_40 (Conv1D)           (None, 100, 474)          674502    
_________________________________________________________________
leaky_re_lu_169 (LeakyReLU)  (None, 100, 474)          0         
_________________________________________________________________
conv1d_41 (Conv1D)           (None, 100, 474)          674502    
_________________________________________________________________
leaky_re_lu_170 (LeakyReLU)  (None, 100, 474)          0         
_________________________________________________________________
batch_normalization_33 (Batc (None, 100, 474)        

In [296]:
def gan_model(g_model, d_model):
	# make weights in the discriminator not trainable
	d_model.trainable = False
	# connect them
	model = Sequential()
	# add generator
	model.add(g_model)
	model.add(BatchNormalization())
	# add the discriminator
	model.add(d_model)
	# compile model
	model.compile(loss='binary_crossentropy', optimizer="adam")
	return model

In [297]:
gan_model = gan_model(g, d)
gan_model.summary()

Model: "sequential_69"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
sequential_67 (Sequential)   (None, 100, 109)          641976    
_________________________________________________________________
batch_normalization_34 (Batc (None, 100, 109)          436       
_________________________________________________________________
sequential_68 (Sequential)   (None, 1)                 2902777   
Total params: 3,545,189
Trainable params: 641,028
Non-trainable params: 2,904,161
_________________________________________________________________


In [70]:
# First, the discriminator model is updated for a half batch of real samples, then a half batch of fake samples, 
# together forming one batch of weight updates. The generator is then updated via the combined GAN model. 
# Importantly, the class label is set to 1 or real for the fake samples. This has the effect of updating the generator toward 
# getting better at generating real samples on the next batch.

In [71]:
# evaluate the discriminator, plot generated images, save generator model
def check_performance(epoch, g_model, d_model, x, n_samples=100):
	# prepare real samples
	x_real, y_real = generate_real_samples(x, n_samples)
	# evaluate discriminator on real examples
	_, acc_real = d_model.evaluate(x_real, y_real, verbose=0)
	# prepare fake examples
	x_fake, y_fake = generate_fake_data(x, g, n_samples)
	# evaluate discriminator on fake examples
	_, acc_fake = d_model.evaluate(x_fake, y_fake, verbose=0)
	# summarize discriminator performance
	print('>Accuracy real: %.0f%%, fake: %.0f%%' % (acc_real*100, acc_fake*100))
	# save plot
	#save_plot(x_fake, epoch)
	# save the generator model tile file
	#filename = 'generator_model_%03d.h5' % (epoch+1)
	#g_model.save(filename)

In [72]:
check_performance = check_performance(1, g, d, x, n_samples=100)


>Accuracy real: 40%, fake: 100%


In [298]:
# train the generator and discriminator
def train(x, g_model, d_model, gan_model, y_gan, n_epochs, n_batch):
	batch_per_epoch = int(x.shape[0]/n_batch)
	half_batch = int(n_batch / 2)
	# manually enumerate epochs
	for i in range(n_epochs):
		# enumerate batches over the training set
		for j in range(batch_per_epoch):
			# get randomly selected 'real' samples
			x_real, y_real = generate_real_samples(x, n_samples=half_batch)
			# update discriminator model weights
			d_loss1, _ = d_model.train_on_batch(x_real, y_real)
			# generate 'fake' examples 
			x_fake, y_fake = generate_fake_data(x, g, n_samples=half_batch)
			# update discriminator model weights
			d_loss2, _ = d_model.train_on_batch(x_fake, y_fake)
			# prepare points in latent space as input for the generator
			x_latent = generate_latent_points(x, n_samples=n_batch)
			# create inverted labels for the fake samples
			y_fake_1 = np.ones((n_batch, 1))
			# update the generator via the discriminator's error
			g_loss = gan_model.train_on_batch(x_latent, y_fake_1)
			# summarize loss on this batch
			print('>%d, %d/%d, loss_real=%.3f, loss_fake=%.3f loss_latent=%.3f' %
				(i+1, j+1, batch_per_epoch, d_loss1, d_loss2, g_loss))
		# evaluate the model performance, sometimes
		if (n_epochs+1) % 10 == 0:
			check_performance(n_epochs, g_model, d_model, x, n_batch)

In [299]:
train(x=x, g_model=g, d_model=d, gan_model=gan_model, y_gan=y_gan, n_epochs=20, n_batch=129)

>1, 1/3, loss_real=0.828, loss_fake=0.762 loss_latent=0.388
>1, 2/3, loss_real=0.000, loss_fake=0.696 loss_latent=0.158
>1, 3/3, loss_real=0.000, loss_fake=0.627 loss_latent=0.027
>2, 1/3, loss_real=0.000, loss_fake=0.548 loss_latent=0.002
>2, 2/3, loss_real=0.000, loss_fake=0.410 loss_latent=0.000
>2, 3/3, loss_real=0.000, loss_fake=0.184 loss_latent=0.000
>3, 1/3, loss_real=0.000, loss_fake=0.017 loss_latent=0.000
>3, 2/3, loss_real=0.000, loss_fake=0.000 loss_latent=0.001
>3, 3/3, loss_real=0.000, loss_fake=0.000 loss_latent=0.005
>4, 1/3, loss_real=0.000, loss_fake=0.000 loss_latent=0.027
>4, 2/3, loss_real=0.000, loss_fake=0.000 loss_latent=0.045
>4, 3/3, loss_real=0.000, loss_fake=0.000 loss_latent=0.006
>5, 1/3, loss_real=0.000, loss_fake=0.000 loss_latent=0.000
>5, 2/3, loss_real=0.000, loss_fake=0.000 loss_latent=0.000
>5, 3/3, loss_real=0.000, loss_fake=0.000 loss_latent=0.000
>6, 1/3, loss_real=0.000, loss_fake=0.000 loss_latent=0.000
>6, 2/3, loss_real=0.000, loss_fake=0.00

In [300]:
# save the model
g.save(path_3 + "model_g_1con.h5")


In [301]:
# load the model 
# model_g_4 - gan con lstm 3 - 100 epochs, 139 batches
# model_g_1con - gan con con 3 - 100 epochs, 139 batches
model_gan_1con = tf.keras.models.load_model(path_3 + "model_g_1con.h5") # 3 - 100 epochs, 139 batches


In [54]:
#gan_model_1 = tf.keras.models.load_model(path_3 + "model_g_3.h5") 


In [313]:
def generate_notes(notes, model, temperature=1.0):
    """ 
    Generate notes from the neural network based on a sequence of notes 
    """
    # pick a random sequence from the input as a starting point for the prediction
    start = np.random.randint(0, len(notes)-100-1)

    pitchnames = sorted(set(notes))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames)) 
    int_to_note = dict((number, note) for number, note in enumerate(pitchnames))
    
    pattern = notes[start: (start+100)] 
    prediction_output = []
    patterns = []

    # generate 500 notes, roughly two minutes of music
    for note_index in range(100):
        prediction_input = np.zeros((1, 100, len(pitchnames)))
        for j, note in enumerate(pattern):
            prediction_input[0, j, note_to_int[note]] = 1.0
        preds = model.predict(prediction_input, verbose=0)[0]
        for elem in list(preds):
            next_index = sample(elem, temperature=temperature)
            next_note = int_to_note[next_index]

        pattern = pattern[1:]
        pattern.append(next_note)

        prediction_output.append(next_note)

        patterns.append(next_index)
        #patterns = patterns[1:len(patterns)]

    return prediction_output, patterns

In [315]:
prediction_output, patterns = generate_notes(load_chopin, model_gan_1con, temperature=1)
print(prediction_output)

['D6', '2.6', 'E-5', '9.0.4', '1.5', 'F#5', '1.7', '2.4.8', '5.9.11', '4.7.9', '2.6', '6.9', '5.11', '11.2', '2.4.6.10', '9.11', 'C2', '9.11', 'B1', '10.0', '5.10', 'D3', 'G2', 'F4', '5.9.11', 'C2', '8.11.2', 'E4', '9', '5.7', 'C#4', '4.8', 'C3', 'E5', '9.0', '1.2', 'E6', 'C6', 'G#2', '0.4.6', 'E-5', 'B5', 'G2', 'A4', 'C6', 'D4', 'E3', 'B2', 'G#3', 'B-4', '4.7.10.0', 'A2', 'C3', '9.0.4', '0.4.6', 'E-4', 'A4', 'C#6', 'F5', 'D5', '2.7', '8.9', '8.11.2.4', 'A2', 'E1', '10.0', '9.0.3', 'NULL', '9.0.3', 'C4', '3.9', 'E-4', 'E-3', '10.2', '5.10', '9', '8.9', '9', '5.9.11', 'G#2', '11.3', 'G#4', 'E3', 'B5', 'E-5', 'E-3', '3.9', '11.2.6', '1.6', '5.9.11', '2.4.6.10', '1.7', 'C#4', 'E1', 'G2', '9', 'G2', '4.7.10.0', '9.0', '1.5']


# Output

In [320]:
def create_midi(prediction_output, patterns, path):
    """ convert the output from the prediction to notes and create a midi file from the notes"""
    
    offset = 0
    output_notes = []

    # create note and chord objects based on the values generated by the model
    for pattern in prediction_output:
        # pattern is a chord
        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                new_note = note.Note(int(current_note))
                new_note.storedInstrument = instrument.Piano()
                notes.append(new_note)
            new_chord = chord.Chord(notes)
            new_chord.offset = offset
            output_notes.append(new_chord)
        # pattern is a rest
        elif ("NULL" in pattern):
            new_rest = note.Rest(pattern)
            output_notes.append(new_rest)
        # pattern is a note
        else:
            new_note = note.Note(pattern)   
            new_note.offset = offset
            new_note.storedInstrument = instrument.Piano()
            output_notes.append(new_note)

        # increase offset each iteration so that notes do not stack
        offset += 0.5

    midi_stream = stream.Stream(output_notes)

    midi_stream.write("midi", fp= path + "test_output_1ganconv.mid")   # first output 01/07/2021

    return midi_stream

In [321]:
create_midi = create_midi(prediction_output, patterns, path_4)
# 13gen - 100 epoch, 129 batches - lstm
# test_output_1ganconv - 100 epoch, 129 batch - conv

In [322]:
def play_music(music_file):
    """
    Play music given a midi file path
    """
    import music21
    try:
        # allow to stop the piece 
        pygame.mixer.init()
        clock = pygame.time.Clock() 
        pygame.mixer.music.load(music_file)
        pygame.mixer.music.play()
        while pygame.mixer.music.get_busy():
            # check if playback has finished
            clock.tick(10)

        freq = 44100    # audio CD quality
        bitsize = -16   # unsigned 16 bit
        channels = 2    # 1 is mono, 2 is stereo
        buffer = 1024    # number of samples
        pygame.mixer.init(freq, bitsize, channels, buffer)

    except KeyboardInterrupt:
        while True:
            action = input('Enter Q to Quit, Enter to Skip.').lower()
            if action == 'q':
                pygame.mixer.music.fadeout(1000)
                pygame.mixer.music.stop()
            else:
                break

In [324]:
# Plays music when the cell is executed 

play_music(path_4 + "test_output_1ganconv.mid")